In [3]:
import pandas as pd
import pickle
from UMLS import UMLS
from tqdm import tqdm
import numpy as np

In [4]:
umls = UMLS()
umls.load_umls()

15229292it [01:00, 349008.62it/s]                                                                                                      
  6%|█████                                                                               | 913160/15229292 [00:06<01:28, 162475.22it/s]


 12%|█████████▉                                                                         | 1829578/15229292 [00:13<01:25, 156081.77it/s]


 18%|██████████████▊                                                                    | 2714701/15229292 [00:19<01:29, 139492.03it/s]


 24%|███████████████████▊                                                                | 3582903/15229292 [00:30<05:09, 37638.59it/s]


 29%|████████████████████████▎                                                          | 4450443/15229292 [00:41<01:33, 115431.60it/s]


...

 87%|█████████████████████████████████████████████████████████████████████████▍          | 13309239/15229292 [02:47<07:28, 4283.63it/s]


{'AUI': 'A27041938', 'CUI': 'C4242292', 'STR': 'Superior facet of posterosuperior zone of greater tubercle of humerus', 'SCUI': '298544|||FMA', 'SOURCE': 'FMA', 'PREF': ('S', 'PF'), 'LANG': 'ENG'}


 92%|███████████████████████████████████████████████████████████████████████████▌      | 14040934/15229292 [02:53<00:10, 115400.27it/s]


 97%|███████████████████████████████████████████████████████████████████████████████▏  | 14709804/15229292 [03:00<00:04, 110637.21it/s]


100%|███████████████████████████████████████████████████████████████████████████████████| 15229292/15229292 [03:04<00:00, 82500.48it/s]
15229292it [04:07, 61475.10it/s] 


In [5]:
aui_dataset = pickle.load(open('/data/Bodenreider_UMLS_DL/Interns/Bernal/aui_pairwise_data_splits.100.p','rb'))

In [6]:
for split, tups in aui_dataset.items():
    
    one_way = []
    two_way = []
    
    if split == 'val':
        split = 'dev'
        
    for aui1, aui2, label in tqdm(tups):

        str1 = umls.aui2str[aui1]
        str2 = umls.aui2str[aui2]

        one_way.append((str1 + ' [SEP] ' + str2, label))
        if split == 'train':
            two_way.append((str1 + ' [SEP] ' + str2, label))
            two_way.append((str2 + ' [SEP] ' + str1, label))
        
    one_way_df = pd.DataFrame(one_way, columns=['sents','labels'])
    one_way_df = one_way_df.sample(len(one_way_df), random_state=np.random.RandomState(42))
    
    if split == 'train':
        two_way_df = pd.DataFrame(two_way, columns=['sents','labels'])
        two_way_df = two_way_df.sample(len(two_way_df), random_state=np.random.RandomState(42))
    else:
        two_way_df = one_way_df
    
    one_way_df.to_csv('/data/Bodenreider_UMLS_DL/Interns/Bernal/data/RW-UVA-2020AB-one-way/{}.tsv'.format(split), sep='\t', quoting=3)
    two_way_df.to_csv('/data/Bodenreider_UMLS_DL/Interns/Bernal/data/RW-UVA-2020AB-two-way/{}.tsv'.format(split), sep='\t', quoting=3)

100%|████████████████████████████████████████████████████████████████████████████████████| 5469322/5469322 [00:11<00:00, 466585.80it/s]
